## Classifying Hourly Timestamps into Day or Night

In [1]:
import pandas as pd
import numpy as np
import warnings
from df2gspread import df2gspread as d2g
warnings.simplefilter('ignore') 

In [2]:
## Import Raw TTN Data from Google SpreadSheet
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRlXVQ6c3fKWvtQlFRSRUs5TI3soU7EghlypcptOM8paKXcUH8HjYv90VoJBncuEKYIZGLq477xE58C/pub?gid=0&single=true&output=csv'
df_hourly = pd.read_csv(url,parse_dates = ['time'],infer_datetime_format = True,usecols = [0,3])
df_hourly.head()

,CarCount,time
0,87,2018-02-21 09:30:20.546748754
1,59,2018-02-21 10:30:36.766391925
2,66,2018-02-21 11:30:52.944428596
3,68,2018-02-21 12:31:07.516473865
4,72,2018-02-21 13:31:22.445650218


In [3]:
## Cleaning and re-organizing the DataFrame
df_hourly.rename(columns={'time': 'TimeStamps'}, inplace=True)
df_hourly.rename(columns={'CarCount': 'VehicleCountperHour'}, inplace=True)

## Strip the Microseconds from the time column
df_hourly['TimeStamps'] = df_hourly['TimeStamps'].values.astype('datetime64[s]')

## Reorder the dataframe
df_hourly = df_hourly.reindex(['TimeStamps','VehicleCountperHour'], axis=1)
df_hourly.tail()

,TimeStamps,VehicleCountperHour
76,2018-02-25 04:05:22,5
77,2018-02-25 05:05:38,16
78,2018-02-25 06:05:53,39
79,2018-02-25 07:06:10,57
80,2018-02-25 08:06:25,47


### Importing SunTime Chart for adding Day or Night Classification
 Here we add Day or Night classification to Hourly Vehicle Count Data using sunrise and sunset times dataframe of Saarbrucken. These times were acquired from [timeanddate.com](https://www.timeanddate.com/sun/germany/saarbrucken) and the dataframe was manually created.

In [4]:
# Reading SunTime Chart of Saarbrücken
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRJCZQkgUHTCcx-wvJOog87qq4EFiQ1W6T4akxLSpiqCb3KjYDf_43coltDGG0YcjjsDTxjeXE-O_NH/pub?gid=0&single=true&output=csv'
df_suntimes = pd.read_csv(url)
# Modified Sun Timings DataFrame
df_suntimes_mod = pd.DataFrame()
df_suntimes_mod['SunriseTimeStamp']= pd.to_datetime(df_suntimes['Date'] + ' ' + df_suntimes['Sunrise'])
df_suntimes_mod['SunsetTimeStamp']= pd.to_datetime(df_suntimes['Date'] + ' ' + df_suntimes['Sunset'])
# Querying values in the dataframe to selected dates
start_time = '2018-02-21 07:25:00'
df_suntimes_mod = df_suntimes_mod.loc[df_suntimes_mod.SunriseTimeStamp >= start_time,:]
end_time = '2018-02-25 18:15:00'
df_suntimes_mod = df_suntimes_mod.loc[df_suntimes_mod.SunriseTimeStamp <= end_time,:]
df_suntimes_mod = df_suntimes_mod.reset_index()
df_suntimes_mod = df_suntimes_mod.drop(['index'], 1)
df_suntimes_mod

,SunriseTimeStamp,SunsetTimeStamp
0,2018-02-21 07:30:00,2018-02-21 18:01:00
1,2018-02-22 07:28:00,2018-02-22 18:03:00
2,2018-02-23 07:26:00,2018-02-23 18:05:00
3,2018-02-24 07:24:00,2018-02-24 18:06:00
4,2018-02-25 07:22:00,2018-02-25 18:08:00


In [5]:
## Creating a new Dataframe from original dataframe to classify Hourly TimeStamps as 'Day' or 'Night':
df_dn = df_hourly

## Set Everything to Day First
df_dn['DayorNight'] = 'Day'

## Manually fixing first day's night timestamps to 'Night'
night_index = (df_dn.loc[df_dn.TimeStamps <= (df_suntimes_mod['SunriseTimeStamp'][0]),:]).index

# Select Night Time Traffic Only from Sunset today to next day Sunrise
n_days = len(df_suntimes_mod['SunriseTimeStamp'])
for i,j in zip(range(n_days),range(1,n_days)):        
    start_time = df_suntimes_mod['SunsetTimeStamp'][i]
    end_time = df_suntimes_mod['SunriseTimeStamp'][j]
    data = df_dn[(df_dn['TimeStamps'] > start_time) & (df_dn['TimeStamps'] < end_time)]
    night_index = night_index.append(data.index)
    
## Set all the Night TimeStamps to 'Night
df_dn['DayorNight'].iloc[night_index] = 'Night'
df_dn.head(15)

,TimeStamps,VehicleCountperHour,DayorNight
0,2018-02-21 09:30:20,87,Day
1,2018-02-21 10:30:36,59,Day
2,2018-02-21 11:30:52,66,Day
3,2018-02-21 12:31:07,68,Day
4,2018-02-21 13:31:22,72,Day
5,2018-02-21 14:31:36,144,Day
6,2018-02-21 15:31:50,81,Day
7,2018-02-21 16:32:05,52,Day
8,2018-02-21 17:32:20,47,Day
9,2018-02-21 18:32:36,68,Night


In [6]:
# Writing the file as csv
df_dn.to_csv('data/DayorNight.csv', date_format="%d/%m/%Y %H:%M:%S",index=False)

In [7]:
## Write pandas dataframe to a Google Sheet Using df2spread:

# Insert ID of Google Spreadsheet
spreadsheet = '1LTXIPNb7MX0qEOU_DbBKC-OwE080kyRvt-i_ejFM-Yg'

# Insert Sheet Name
wks_name = 'CleanedData'

d2g.upload(df_dn,spreadsheet,wks_name,col_names=True,clean=True)

<Worksheet 'CleanedData' id:oa6s1mq>